# Minutely RSI Strategy Research & Analysis

![QuantConnect Logo](https://cdn.quantconnect.com/web/i/icon.png)

## Strategy Overview

This notebook analyzes the Minutely RSI Trading Strategy using minute-level data on SPY.

**Strategy Configuration:**
- **Data Resolution**: MINUTE
- **RSI Period**: 14 (default, optimizable)
- **Oversold Threshold**: 30 (optimizable: 20-40)
- **Overbought Threshold**: 70 (optimizable: 60-80)
- **Buy Signal**: When RSI crosses below oversold threshold
- **Sell Signal**: When RSI crosses above overbought threshold
- **Position Size**: 95% of available capital
- **Stop Loss**: 2% per trade
- **Take Profit**: 4% per trade

## Research Objectives

1. Analyze RSI behavior on minute-level data
2. Optimize RSI parameters (period, thresholds)
3. Evaluate signal quality and frequency
4. Compare with other technical indicators
5. Assess transaction cost impact
6. Develop risk management improvements


In [ ]:
# Initialize QuantBook for analysis
qb = QuantBook()

# Add SPY equity with MINUTE resolution
spy = qb.add_equity("SPY", Resolution.MINUTE)

# Set analysis period
qb.set_start_date(2023, 1, 1)
qb.set_end_date(2024, 1, 1)

# Get minute-level data for SPY
history = qb.history(qb.securities.keys(), Resolution.MINUTE)

print(f"Data range: {history.index[0]} to {history.index[-1]}")
print(f"Total data points: {len(history)}")
print(f"Starting price: ${history['close'].iloc[0]:.2f}")
print(f"Ending price: ${history['close'].iloc[-1]:.2f}")
print(f"Price change: {((history['close'].iloc[-1] / history['close'].iloc[0]) - 1) * 100:.2f}%")
print(f"High price: ${history['high'].max():.2f}")
print(f"Low price: ${history['low'].min():.2f}")
print(f"Average volume: {history['volume'].mean():.0f}")

# Calculate basic statistics
daily_returns = history['close'].resample('D').last().pct_change().dropna()
print(f"Daily volatility: {daily_returns.std():.4f}")
print(f"Annualized volatility: {daily_returns.std() * np.sqrt(252):.2%}")


## RSI Calculation and Analysis

Let's calculate the RSI for different periods and analyze its behavior.


In [ ]:
# Calculate RSI for different periods
import numpy as np
import pandas as pd

def calculate_rsi(prices, period=14):
    """Calculate RSI using exponential moving average"""
    delta = prices.diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=period).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=period).mean()
    rs = gain / loss
    rsi = 100 - (100 / (1 + rs))
    return rsi

# Calculate RSI for different periods
periods = [7, 14, 21, 28]
rsi_data = {}

for period in periods:
    rsi_data[f'RSI_{period}'] = calculate_rsi(history['close'], period)

# Add RSI data to history DataFrame
for column, rsi_values in rsi_data.items():
    history[column] = rsi_values

# Display RSI statistics
for period in periods:
    rsi_col = f'RSI_{period}'
    if rsi_col in history.columns:
        rsi_series = history[rsi_col].dropna()
        print(f"\nRSI {period} Statistics:")
        print(f"  Mean: {rsi_series.mean():.2f}")
        print(f"  Std: {rsi_series.std():.2f}")
        print(f"  Min: {rsi_series.min():.2f}")
        print(f"  Max: {rsi_series.max():.2f}")
        print(f"  % Time in Oversold (<30): {(rsi_series < 30).mean():.1%}")
        print(f"  % Time in Overbought (>70): {(rsi_series > 70).mean():.1%}")
        print(f"  % Time in Neutral (30-70): {((rsi_series >= 30) & (rsi_series <= 70)).mean():.1%}")


## Signal Generation Analysis

Let's analyze the signal generation for different RSI configurations.


In [ ]:
def generate_rsi_signals(data, rsi_col, oversold=30, overbought=70, min_interval=5):
    """Generate buy/sell signals based on RSI with minimum interval constraint"""
    signals = []
    last_signal_time = None
    
    for i in range(1, len(data)):
        current_time = data.index[i]
        current_price = data['close'].iloc[i]
        current_rsi = data[rsi_col].iloc[i]
        
        # Check minimum interval
        if last_signal_time:
            time_diff = (current_time - last_signal_time).total_minutes()
            if time_diff < min_interval:
                continue
        
        # Skip if RSI is not ready
        if pd.isna(current_rsi):
            continue
        
        # Generate signals
        prev_rsi = data[rsi_col].iloc[i-1]
        
        # Buy signal: RSI crosses below oversold threshold
        if (current_rsi <= oversold and prev_rsi > oversold):
            signals.append({
                'time': current_time,
                'price': current_price,
                'rsi': current_rsi,
                'signal': 'BUY',
                'strength': (oversold - current_rsi) / oversold
            })
            last_signal_time = current_time
        
        # Sell signal: RSI crosses above overbought threshold
        elif (current_rsi >= overbought and prev_rsi < overbought):
            signals.append({
                'time': current_time,
                'price': current_price,
                'rsi': current_rsi,
                'signal': 'SELL',
                'strength': (current_rsi - overbought) / (100 - overbought)
            })
            last_signal_time = current_time
    
    return pd.DataFrame(signals)

# Analyze signals for different RSI periods and thresholds
rsi_periods = [14, 21]
threshold_combinations = [
    (25, 75), (30, 70), (35, 65)
]

signal_analysis = []

for period in rsi_periods:
    rsi_col = f'RSI_{period}'
    if rsi_col not in history.columns:
        continue
    
    for oversold, overbought in threshold_combinations:
        signals_df = generate_rsi_signals(history, rsi_col, oversold, overbought)
        
        if len(signals_df) > 0:
            buy_signals = signals_df[signals_df['signal'] == 'BUY']
            sell_signals = signals_df[signals_df['signal'] == 'SELL']
            
            signal_analysis.append({
                'RSI_Period': period,
                'Oversold': oversold,
                'Overbought': overbought,
                'Total_Signals': len(signals_df),
                'Buy_Signals': len(buy_signals),
                'Sell_Signals': len(sell_signals),
                'Signal_Rate_per_Day': len(signals_df) / ((history.index[-1] - history.index[0]).days)
            })

# Create signal analysis DataFrame
signal_df = pd.DataFrame(signal_analysis)
print("Signal Analysis Summary:")
print(signal_df.to_string(index=False))


## Visualization

Let's create visualizations to better understand the RSI behavior and signals.


In [ ]:
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib.patches import Rectangle

# Create subplots
fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(15, 12), sharex=True)

# Plot 1: Price and SMA
ax1.plot(history.index, history['close'], label='SPY Price', linewidth=1, color='blue')
ax1.set_ylabel('Price ($)', fontsize=12)
ax1.set_title('SPY Price and Technical Indicators', fontsize=14, fontweight='bold')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Plot 2: RSI
ax2.plot(history.index, history['RSI_14'], label='RSI (14)', linewidth=1, color='purple')
ax2.axhline(y=30, color='green', linestyle='--', alpha=0.7, label='Oversold (30)')
ax2.axhline(y=70, color='red', linestyle='--', alpha=0.7, label='Overbought (70)')
ax2.axhline(y=50, color='gray', linestyle='-', alpha=0.5, label='Neutral (50)')
ax2.fill_between(history.index, 30, 70, alpha=0.1, color='gray', label='Neutral Zone')
ax2.fill_between(history.index, 0, 30, alpha=0.1, color='red', label='Oversold Zone')
ax2.fill_between(history.index, 70, 100, alpha=0.1, color='red', label='Overbought Zone')
ax2.set_ylabel('RSI', fontsize=12)
ax2.set_title('RSI (14) with Trading Zones', fontsize=12)
ax2.set_ylim(0, 100)
ax2.legend(loc='upper left', fontsize=10)
ax2.grid(True, alpha=0.3)

# Plot 3: Volume
ax3.bar(history.index, history['volume'], alpha=0.6, color='orange', width=1)
ax3.set_ylabel('Volume', fontsize=12)
ax3.set_xlabel('Date', fontsize=12)
ax3.set_title('Trading Volume', fontsize=12)
ax3.grid(True, alpha=0.3)

# Format x-axis
for ax in [ax1, ax2, ax3]:
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))
    ax.xaxis.set_major_locator(mdates.WeekdayLocator(interval=2))
    plt.setp(ax.xaxis.get_majorticklabels(), rotation=45)

plt.tight_layout()
plt.show()


## Backtesting Framework

Let's implement a basic backtesting framework to evaluate the RSI strategy.


In [ ]:
class RSIBacktester:
    """RSI Strategy Backtester"""
    
    def __init__(self, data, initial_capital=100000):
        self.data = data.copy()
        self.initial_capital = initial_capital
        self.reset()
    
    def reset(self):
        """Reset backtest state"""
        self.portfolio = self.initial_capital
        self.position = 0  # 0 = cash, 1 = long
        self.entry_price = 0
        self.trades = []
        self.equity_curve = []
        self.daily_returns = []
    
    def run_backtest(self, rsi_col, oversold=30, overbought=70, 
                     stop_loss_pct=0.02, take_profit_pct=0.04, 
                     max_position=0.95):
        """Run RSI strategy backtest"""
        self.reset()
        
        for i in range(1, len(self.data)):
            current_time = self.data.index[i]
            current_price = self.data['close'].iloc[i]
            current_rsi = self.data[rsi_col].iloc[i]
            
            # Skip if RSI is not available
            if pd.isna(current_rsi):
                self.equity_curve.append(self.portfolio)
                continue
            
            prev_rsi = self.data[rsi_col].iloc[i-1]
            
            # Check position-based exits first (stop-loss, take-profit)
            if self.position == 1:
                price_change_pct = (current_price - self.entry_price) / self.entry_price
                
                # Stop-loss check
                if price_change_pct <= -stop_loss_pct:
                    self._close_position(current_time, current_price, 'STOP_LOSS')
                # Take-profit check
                elif price_change_pct >= take_profit_pct:
                    self._close_position(current_time, current_price, 'TAKE_PROFIT')
            
            # Entry signals (only if not in position)
            if self.position == 0:
                # Buy signal: RSI crosses below oversold
                if (current_rsi <= oversold and prev_rsi > oversold):
                    self._open_position(current_time, current_price)
            
            elif self.position == 1:
                # Sell signal: RSI crosses above overbought
                if (current_rsi >= overbought and prev_rsi < overbought):
                    self._close_position(current_time, current_price, 'RSI_SIGNAL')
            
            # Record equity
            if self.position == 1:
                current_value = self.shares * current_price
            else:
                current_value = self.portfolio
            
            self.equity_curve.append(current_value)
    
    def _open_position(self, time, price):
        """Open long position"""
        position_value = self.portfolio * 0.95  # Use 95% of capital
        self.shares = int(position_value / price)
        self.portfolio = self.portfolio - (self.shares * price)
        self.position = 1
        self.entry_price = price
    
    def _close_position(self, time, price, reason):
        """Close long position"""
        proceeds = self.shares * price
        pnl = proceeds - (self.shares * self.entry_price)
        self.portfolio = self.portfolio + proceeds
        
        # Record trade
        self.trades.append({
            'entry_time': self.last_entry_time,
            'exit_time': time,
            'entry_price': self.entry_price,
            'exit_price': price,
            'shares': self.shares,
            'pnl': pnl,
            'pnl_pct': pnl / (self.shares * self.entry_price),
            'reason': reason
        })
        
        self.position = 0
        self.entry_price = 0
        self.shares = 0
    
    def get_performance_metrics(self):
        """Calculate performance metrics"""
        if len(self.equity_curve) == 0:
            return {}
        
        final_value = self.equity_curve[-1]
        total_return = (final_value / self.initial_capital) - 1
        
        # Calculate daily returns
        equity_series = pd.Series(self.equity_curve, index=self.data.index[:len(self.equity_curve)])
        daily_returns = equity_series.resample('D').last().pct_change().dropna()
        
        # Performance metrics
        metrics = {
            'total_return': total_return,
            'annual_return': (1 + total_return) ** (252 / len(daily_returns)) - 1,
            'volatility': daily_returns.std() * np.sqrt(252),
            'sharpe_ratio': (daily_returns.mean() * 252) / (daily_returns.std() * np.sqrt(252)) if daily_returns.std() > 0 else 0,
            'max_drawdown': self._calculate_max_drawdown(),
            'total_trades': len(self.trades),
            'win_rate': len([t for t in self.trades if t['pnl'] > 0]) / len(self.trades) if self.trades else 0
        }
        
        return metrics
    
    def _calculate_max_drawdown(self):
        """Calculate maximum drawdown"""
        equity_series = pd.Series(self.equity_curve)
        peak = equity_series.expanding().max()
        drawdown = (equity_series - peak) / peak
        return drawdown.min()

# Run backtest with default parameters
backtester = RSIBacktester(history)
backtester.run_backtest('RSI_14', oversold=30, overbought=70)

metrics = backtester.get_performance_metrics()
print("Backtest Results (RSI 14, 30/70 thresholds):")
print(f"  Total Return: {metrics['total_return']:.2%}")
print(f"  Annual Return: {metrics['annual_return']:.2%}")
print(f"  Volatility: {metrics['volatility']:.2%}")
print(f"  Sharpe Ratio: {metrics['sharpe_ratio']:.2f}")
print(f"  Max Drawdown: {metrics['max_drawdown']:.2%}")
print(f"  Total Trades: {metrics['total_trades']}")
print(f"  Win Rate: {metrics['win_rate']:.1%}")


## Parameter Optimization

Let's optimize the RSI parameters to find the best configuration.


In [ ]:
# Parameter optimization
rsi_periods = [10, 14, 21, 28]
oversold_levels = [20, 25, 30, 35]
overbought_levels = [65, 70, 75, 80]

optimization_results = []

print("Optimizing RSI Strategy Parameters...")
print("Period | Oversold | Overbought | Total Return | Sharpe | Max DD | Trades | Win Rate")
print("-" * 80)

for period in rsi_periods:
    rsi_col = f'RSI_{period}'
    if rsi_col not in history.columns:
        continue
    
    for oversold in oversold_levels:
        for overbought in overbought_levels:
            if oversold >= overbought:  # Ensure oversold < overbought
                continue
            
            # Run backtest
            bt = RSIBacktester(history)
            bt.run_backtest(rsi_col, oversold, overbought)
            metrics = bt.get_performance_metrics()
            
            # Store results
            optimization_results.append({
                'rsi_period': period,
                'oversold': oversold,
                'overbought': overbought,
                'total_return': metrics['total_return'],
                'sharpe_ratio': metrics['sharpe_ratio'],
                'max_drawdown': metrics['max_drawdown'],
                'total_trades': metrics['total_trades'],
                'win_rate': metrics['win_rate']
            })
            
            # Print progress
            print(f"{period:6d} | {oversold:8d} | {overbought:10d} | ",
                  f"{metrics['total_return']:11.2%} | {metrics['sharpe_ratio']:6.2f} | ",
                  f"{metrics['max_drawdown']:7.2%} | {metrics['total_trades']:7d} | ",
                  f"{metrics['win_rate']:8.1%}")

# Convert to DataFrame for analysis
opt_df = pd.DataFrame(optimization_results)

# Find best parameters
best_sharpe = opt_df.loc[opt_df['sharpe_ratio'].idxmax()]
best_return = opt_df.loc[opt_df['total_return'].idxmax()]
best_risk_adjusted = opt_df.loc[(opt_df['total_return'] / abs(opt_df['max_drawdown'])).idxmax()]

print(f"\n=== OPTIMIZATION RESULTS ===")
print(f"Best Sharpe Ratio: {best_sharpe['rsi_period']} period, ",
      f"{best_sharpe['oversold']}/{best_sharpe['overbought']} thresholds")
print(f"  Return: {best_sharpe['total_return']:.2%}, Sharpe: {best_sharpe['sharpe_ratio']:.2f}, ",
      f"MDD: {best_sharpe['max_drawdown']:.2%}")

print(f"\nBest Total Return: {best_return['rsi_period']} period, ",
      f"{best_return['oversold']}/{best_return['overbought']} thresholds")
print(f"  Return: {best_return['total_return']:.2%}, Sharpe: {best_return['sharpe_ratio']:.2f}, ",
      f"MDD: {best_return['max_drawdown']:.2%}")

print(f"\nBest Risk-Adjusted: {best_risk_adjusted['rsi_period']} period, ",
      f"{best_risk_adjusted['oversold']}/{best_risk_adjusted['overbought']} thresholds")
print(f"  Return: {best_risk_adjusted['total_return']:.2%}, Sharpe: {best_risk_adjusted['sharpe_ratio']:.2f}, ",
      f"MDD: {best_risk_adjusted['max_drawdown']:.2%}")


## Conclusion and Recommendations

Based on the analysis, here are the key findings and recommendations:

### Key Findings:
1. **RSI Behavior**: RSI oscillates between 0-100, with most time spent in neutral zone
2. **Signal Frequency**: Shorter RSI periods generate more signals but may be noisier
3. **Threshold Impact**: Wider thresholds (20/80) reduce signal frequency but may miss opportunities
4. **Risk Management**: Stop-loss and take-profit are crucial for capital preservation

### Recommendations:
1. **Optimal Period**: Consider RSI(21) for balance between responsiveness and stability
2. **Threshold Settings**: 30/70 thresholds provide good balance of signals
3. **Risk Management**: Implement stop-loss (2%) and take-profit (4%) rules
4. **Signal Filtering**: Add minimum time intervals to prevent over-trading
5. **Market Regime**: Consider market volatility when adjusting parameters

### Next Steps:
1. Implement walk-forward optimization
2. Test on out-of-sample data
3. Add transaction cost modeling
4. Compare with buy-and-hold benchmark
5. Validate with paper trading
